#**Operasi File.csv**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import pickle
import matplotlib.pyplot as plt
import io

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
%cd '/content/drive/MyDrive/SEMESTER 5/1-Capstone Project/2-PSC/Sentiment Analysis'

/content/drive/MyDrive/SEMESTER 5/1-Capstone Project/2-PSC/Sentiment Analysis


In [ ]:
from os import read
import csv
with open('datasetnew.csv') as csvfile:
  readCSV = csv.reader(csvfile, delimiter=',')
  for row in readCSV: #looping perbaris
    print(row)

['Reviews', 'Rating']
['Sebenernya ini aplikasi biasa saja hanya saja, saya mendownload aplikasi ini karena diwajibkan oleh sekolah dan masalahnya aplikasi ini mengambil data pribadi yang tentu saja saya keberatan. Kalau ingin membantu seseorang ayolah membantu dengan ikhlas dan Jagan malah mengambil data pribadi seseorang.', '1']
['Kak maaf mau tanya, kan waktu di halaman yang ada tulisan username/emain dan password kok setelah dimasukan kata katanya ke kolom itu waktu mau daftar kok gak bisa daftar y kak, palah ada tulisan username not found', '1']
['Aplikasikannya belum siap, loadingnya tidak ada ampun... Lama banget, cape nunggunya... Server over load dan belum siap, persiapkan dulu...', '1']
['Aneh padahal daftar sudah benar malah disalahkan,aplikasi tidak bermutu', '1']
['Aplikasi tidak guna hanya loading saja.. Padahal sinyal WiFi kencang', '1']
['Aplikasi ngelag ga bermutu mau dikasih ratting tinggi? Benerin dulu aplikasinya bos, baru sebarkan. Sampah', '1']
['username sudah di

In [ ]:
path_data_sentimen = "/content/drive/MyDrive/SEMESTER 5/1-Capstone Project/2-PSC/Sentiment Analysis/datasetnew.csv"
data_sentiment = pd.read_csv(path_data_sentimen, sep=",", header=[0], encoding="UTF-8")

##Labeling

In [ ]:
data_sentiment.head()

TypeError: ignored

In [ ]:
def pelabelan(rate):
  if rate == 3:
    return 0
  elif rate < 3:
    return -1
  elif rate > 3:
    return 1

data_sentiment['Label'] = data_sentiment['Rating'].apply(pelabelan)
data_sentiment.head()

KeyError: ignored

In [ ]:
data_reviews = data_sentiment['Reviews']
print (data_reviews[:4])

#**Preprocessing**

##Casefolding

Proses mengubah semua karakter huruf pada sebuah kalimat menjadi huruf kecil atau huruf besar. Case folding yang dilakukan yaitu mengubah seluruh dataset menjadi huruf kecil.

In [ ]:
data_casefolding = data_reviews.str.lower()
data_casefolding.head()

##Filtering

Proses dalam menghilangkan karakter-karakter illegal.

Contoh : %, &, >, (,
{, ], 1-9, @uluwatu, http://...

In [ ]:
#filtering url -> menggunakan list comprehension dan menggunakan regular expression re.sub mengganti url dengan spasi
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", ulasan) for ulasan in data_casefolding]

#filtering cont -> menggunakan list comprehension, kata cont(continued) dianggap sebagai noise dalam proses analisis data
filtering_cont = [re.sub(r'\(cont\)'," ", Reviews)for Reviews in filtering_url]

#filtering punctuatuion -> menggunakan list comprehension untuk menghapus tanda baca dari setiap ulasan
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', Reviews) for Reviews in filtering_cont]  #hapus simbol'[!#?,.:";@()-_/\']'

#filtering tagger -> menghapus #tagger dan mengganti tagar dengan spasi
filtering_tagger = [re.sub(r'#([^\s]+)', '', Reviews) for Reviews in filtering_punctuation]

#filtering numeric -> menghapus angka dari setiap ulasan dalam dataset dengan spasi
filtering_numeric = [re.sub(r'\d+', ' ', Reviews) for Reviews in filtering_tagger]

data_filtering = pd.Series(filtering_numeric) #pandas series

print (data_filtering[:4])
#print (data_filtering) #jika ingin mencetak semua data_filtering

Dengan melakukan langkah-langkah diatas,teks telah dibersihkan dari URL, kata "(cont)", tanda baca, tagar, dan angka. Langkah-langkah diatas dapat membantu dalam meningkatkan kualitas data dan mencegah noise yang tidak diinginkan sebelum melakukan analisis lebih lanjut atau membangun model.

##Tokenization

Proses untuk memecah dokumen teks menjadi token. Tokenisasi memiliki kemampuan untuk memecah dokumen menjadi kata, frasa, simbol atau elemen lain yang memiliki makna. Ada 2 cara yaitu word dan tweettokenizer.

In [ ]:
import nltk
nltk.download('punkt') #model tokenisasi kata
from nltk.tokenize import word_tokenize

#tokenisasi kata pada setiap baris
data_tokens = [word_tokenize(line) for line in data_filtering]
print(data_tokens)

##Stopword Removal

Tahap pengambilan kata-kata penting dan membuang kata-kata yang dianggap tidak penting (Cara untuk membuang kata yang tidak penting). Stopword removal bertujuan untuk menghilangkan kata-kata yang sering muncul namun tidak memiliki kontribusi dalam proses analisis data.
Contoh : dan, yang, di, ke, dll

In [ ]:
!pip install sastrawi

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [ ]:
factory = StopWordRemoverFactory()
ind_stopword = factory.get_stop_words()

#penedefinisian fungsi stopword
def stopword(line):
  temp = list()
  for word in line:
    if(len(word)>3):
      temp.append(word)
  return temp

data_katapenting = [stopword (line) for line in data_tokens]
print(data_katapenting)

## Slangword to Standword

Mengganti semua kata slang ke dalam bentuk standarnya dengan proses konversi slang word to standar word.

Contoh : Bkn (slang/tdk baku) Bukan (Baku)

In [ ]:
#slang word
path_dataslang = open("kamus kata baku 1.csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8',header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

reviews_formal = []
for data in data_tokens:
  data_clean = [replaceSlang(word) for word in data]
  reviews_formal.append(data_clean)
len_reviews_formal = len(reviews_formal)
print(reviews_formal)
len_reviews_formal

In [ ]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
data_notstopword = [removeStopWords(line,stopwords) for line in reviews_formal]

##Steamming

Proses stemming dilakukan dengan menghapus imbuhan, baik awalan maupun akhiran dari suatu kata untuk mendapatkan kata dasarnya.

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
factory = StemmerFactory()
ind_stemmer =factory.create_stemmer()

def stemmer(line):
  temp = list()
  for word in line:
      word = ind_stemmer.stem(word)
      temp.append(word)
  return temp

reviews_dasar = [stemmer (line) for line in reviews_formal]

#cetak 2 baris pertama
print(reviews_dasar[:2])

##Lemmatization

Untuk membersihkan dan mengolah ulang (preprocessing). Lemmatization bertujuan untuk mengembalikan kata ke bentuk dasarnya, sementara stemming mencoba untuk menghapus afiks dari kata-kata. (lebih powerfull)

Contohnya, "running" bisa diubah menjadi "run" melalui lemmatization atau stemming.

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
data_sentiment['Reviews'][1]

In [ ]:
#pengembalian kata ke bentuk dasar
lemma = WordNetLemmatizer()
stemmer = PorterStemmer()

def CleanReview(txt):
  txt = re.sub(r'http\S+', ' ', txt) #mengganti semua url dengan spasi
  txt = re.sub(r'[^a-zA-Z]', ' ', txt) #mengganti karakter non-alfabet dengan spasi
  txt = str(txt).lower() #mengubah huruf besar ke kecil
  txt = word_tokenize(txt) #teks menjadi token
  txt = [item for item in txt if item not in data_notstopword] #menghapus kata yg ga dipake (stopword)
  txt = [lemma.lemmatize(word=w, pos='v')for w in txt] #mengembalikan kata ke bentuk dasar
  txt = [i for i in txt if len(i) > 2] #menghapus kata-kata yang terlalu pendek
  txt = ' '.join(txt)
  return txt
data_sentiment['CleanReview'] = data_sentiment['Reviews'].apply(CleanReview)

data_sentiment.head()

Tujuan dari tahap ini adalah untuk mempersiapkan teks agar dapat digunakan dengan lebih baik dalam pemrosesan atau pembuatan model machine learning, dengan menghilangkan noise, mengurangi dimensi, dan meningkatkan kualitas fitur.

#**Training Data and Split Data**

In [ ]:
#Training data x_train, x_test, y_train, y_tes
x = data_sentiment ['CleanReview']
y = data_sentiment ['Label']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=4)
nama = "Model SVM"

#**Feauture Extraction for Mechine Learning (SVM)**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

In [ ]:
# Membuat model SVM dengan TF-IDF sebagai fitur
model_svm = make_pipeline(TfidfVectorizer(), LinearSVC(C=0.75))

In [ ]:
# Melatih model pada data training
model_svm.fit(x_train, y_train)

In [ ]:
# Melakukan prediksi pada data testing
y_pred = model_svm.predict(x_test)

#**Evaluasi Model**

In [ ]:
# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
# Menampilkan hasil evaluasi
print(f"{nama} - Akurasi: {accuracy:.2f}")
print(f"\nClassification Report:\n{classification_rep}")
print(f"\nConfusion Matrix:\n{conf_matrix}")

#**Deployment**

In [ ]:
model = pickle.load(open('/content/drive/MyDrive/SEMESTER 5/1-Capstone Project/2-PSC/Sentiment Analysis/tfidf_'+str(nama)+'_nvb.pkl','rb'))
pred_y_ = model.predict(x_test) #prediksi classification_report
label_test = y_test
print('\n'+'-'*45)
print('confusion matrix : ' + str(nama))
print('-'*45,'\n')
print(classification_report(label_test, pred_y_, labels=[-1, 0, 1]))

In [ ]:
pickle.dump(svm, open('/content/drive/MyDrive/SEMESTER 5/1-Capstone Project/2-PSC/Sentiment Analysis/tfidf_{}_nvb.pkl'.format(nama), 'wb'))